In [1]:
# load packages
import pandas as pd
# !pip install deep_translator
from deep_translator import GoogleTranslator, exceptions
# !pip install luga
from luga import language
import numpy as np
# !pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

### Load and prepare data

In [2]:
# load data
raw = pd.read_parquet('translated.parquet.gzip')

In [3]:
# access your multilingual data and ensure it is lowercase
data = pd.DataFrame()
data['trans_text'] = raw.trans_text.str.lower()

### Translate multilingual tickets into English for processing

In [4]:
def translate_str(text):
    lang = language(text).name
    if lang == 'zh':
        lang = 'zh-CN'
    try:
        trans = GoogleTranslator(source=lang, target='en').translate(text)
    except exceptions.LanguageNotSupportedException:
        trans = text
        
    return trans, lang.upper()

In [ ]:
# translate multilingual to english
data['input_english'] = data.trans_text.apply(translate_str).str[0]

### Create initial urgency ratings based on sentiment and strength of the negative sentiment

In [ ]:
# run sentiment analysis to find negatives and potential negatives
sia = SentimentIntensityAnalyzer()

def v_polarity(text):
    return sia.polarity_scores(text)['compound']

def v_negativity(text):
    return sia.polarity_scores(text)['neg']

data['polarity'] = data['en_text'].apply(v_polarity)
data['negativity'] = data['en_text'].apply(v_negativity)

In [ ]:
# label tickets as positive, negative, or neutral
min_positive = 0.3
min_neutral = 0

data['sentiment'] = np.select([((data['polarity'] > min_positive) & ((data['negativity'] == 0) | (data['polarity'] > 0.7))),
                                ((data['polarity'] < min_neutral) | (data['negativity'] > 0)),
                                ((min_positive >= data['polarity']) & (data['polarity'] >= min_neutral))],
                                ['Positive', 'Negative', 'Neutral'])

In [ ]:
# classify urgency based on level of negativity
high_urgency_max = -0.5
mid_urgency_max = -0.35
low_urgency_min = -0.2

data['urgency_polarity'] = np.select([(data['polarity'] < high_urgency_max),
                                      ((data['polarity'] > low_urgency_min) & (data['sentiment'] != "Positive")),
                                      ((mid_urgency_max <= data['polarity']) & (data['polarity'] <= low_urgency_min)),
                                      ((high_urgency_max <= data['polarity']) & (data['polarity'] <= mid_urgency_max))],
                                     [4, 1, 2, 3],
                                     0).astype(int)

### Adjust urgency rating based on words and characters used in the ticket

In [ ]:
# keywords for urgency; score based on number of words in text
urgent_words = ['need', 'urgent', 'urgency', 'urgently', 'please', 'help', 'useless', 'immediate', 'immediately',
                'dire', 'asap', 'pay', 'paid', 'worst', 'worse', 'terrible', 'terribly', 'broke', 'broken',
                'disappoint', 'disappointed', 'disappointingly', 'disappointing', 'quick', 'fast',
                'dying', 'dead', 'death', 'kill', 'important', 'serious', 'pressing', 'now']
urgent_symbols = ['!', '$', '?']


def urgent_word_count(text):
    count = 0
    for symbol in urgent_symbols:
        count += text.count(symbol)

    text = ''.join([a for a in text if a.isalpha() or a == " "])

    to_check = text.split()
    for word in urgent_words:
        count += to_check.count(word)

    return count

In [ ]:
# get counts of urgent text
data['urgency_text'] = data['input_english'].apply(urgent_word_count)

In [ ]:
# set levels for increasing urgency based on text counts
if len(data) > 20:
    urgency_text_level2 = data.urgency_text.iloc[round(len(data.urgency_text) * .1)]
    urgency_text_level1 = data.urgency_text.iloc[round(len(data.urgency_text) * .25)]
else:
    urgency_text_level2 = 4
    urgency_text_level1 = 2

In [ ]:
# increase urgency level based on category and text count score
data['rating'] = np.where(((data['urgency_text'] > urgency_text_level2) & (data['urgency_polarity'] > 0)),
                                      data['urgency_polarity'] + 1,
                                      data['urgency_polarity'])

data['rating'] = np.where(((data['urgency_text'] > urgency_text_level1) & (data['urgency_polarity'] > 0)),
                                  data['rating'] + 1,
                                  data['rating'])

data['rating'] = np.where((data['polarity'] == 'Positive'),
                                  data['rating'] * 2,
                                  data['rating'])

data['rating'] = np.where((data['polarity'] == 'Neutral'),
                                  (data['rating']-1)*2 + 1,
                                  data['rating'])

### Create final urgency classification based on above

In [ ]:
# create classification based on urgency ratings
data['classification'] = np.select([(data['rating'] == 6),
                                        (data['rating'] == 5),
                                        (data['rating'] == 4),
                                        (data['rating'] == 3),
                                        (data['rating'] == 2),
                                        ((data['rating'] == 1) & (data['urgency_text'] > 0)),
                                        ((data['rating'] == 1) & (data['urgency_text'] == 0))],
                                       ['HIGH', 'HIGH', 'Mid-High', 'Low-Mid', 'Low', 'Lowest', 'No Negativity or Urgency'],
                                       'No Negativity or Urgency')

In [ ]:
# make urgency classes sortable
data.classification = pd.Categorical(data.classification,
                                         categories=['No Negativity or Urgency', 'Lowest', 'Low', 'Low-Mid', 'Mid-High', 'HIGH'],
                                         ordered=True)
data.sentiment = pd.Categorical(data.sentiment,
                                categories=['Negative', 'Neutral', 'Positive'],
                                ordered=True)

data.sort_values(by='rating', ascending=False, inplace=True)
data = data[['input', 'input_english', 'classification', 'sentiment', 'negativity', 'rating']]
data.columns = data.columns.str.capitalize()

### View the final dataset to evaluate tickets based on final classification

In [ ]:
data